In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_grid= {'n_neighbors':[10,50,100], 
             'metric': ['minkowski'], 
             'weights':['uniform', 'distance'],
             'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
             'p':[2]}
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'], 'n_neighbors': [10, 50, 100],
                         'p': [2], 'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [57]:
grid_predictions=grid.predict(X_test)


from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_predictions)

from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_predictions)

In [58]:
from sklearn.metrics import f1_score
f1_micro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_micro value for best parameter{}:".format(grid.best_params_),f1_micro)

The f1_micro value for best parameter{'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}: 0.9252129537257311


In [59]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])


0.9612534732942266

In [60]:
re=grid.cv_results_
print("The re for the best Parameter{}:".format(grid.best_params_),re)


The re for the best Parameter{'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}: {'mean_fit_time': array([0.01099291, 0.01239209, 0.00699496, 0.01039352, 0.00759568,
       0.01359267, 0.00959425, 0.01798863, 0.01419163, 0.00719976,
       0.00699735, 0.01659069, 0.00719643, 0.01199198, 0.00759454,
       0.00759573, 0.00839448, 0.00579591, 0.00659504, 0.0053967 ,
       0.00559564, 0.00419636, 0.00379767, 0.00519671]), 'std_fit_time': array([0.00419286, 0.01035905, 0.00167286, 0.00504048, 0.00135638,
       0.00957792, 0.0045835 , 0.01339327, 0.01095548, 0.0007447 ,
       0.00178966, 0.01097654, 0.00097937, 0.00981199, 0.00196083,
       0.00048955, 0.00185357, 0.00074883, 0.00135649, 0.00048912,
       0.00195915, 0.00116607, 0.00074805, 0.00116521]), 'mean_score_time': array([0.00899549, 0.00419707, 0.00979457, 0.00559659, 0.01219153,
       0.00559511, 0.01039453, 0.00459743, 0.01119165, 0.00439353,
       0.00939336, 0.00839415, 0.009792

In [61]:
table=pd.DataFrame.from_dict(re)

In [62]:
print(cm)

[[74  5]
 [ 4 37]]


In [63]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010993,0.004193,0.008995,3.030541e-03,auto,minkowski,10,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.855314,0.874254,0.858503,0.946663,0.964286,0.899804,0.046242,1
1,0.012392,0.010359,0.004197,9.802605e-04,auto,minkowski,10,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.855314,0.874254,0.859435,0.929144,0.964286,0.896486,0.042955,5
2,0.006995,0.001673,0.009795,1.326058e-03,auto,minkowski,50,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.729723,0.723577,0.588177,0.830519,0.708333,0.716066,0.077146,17
3,0.010394,0.005040,0.005597,2.243617e-03,auto,minkowski,50,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.835985,0.855314,0.802399,0.910254,0.888158,0.858422,0.038007,9
4,0.007596,0.001356,0.012192,2.037725e-03,auto,minkowski,100,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.674877,0.673469,0.588177,0.718293,0.590909,0.649145,0.051271,21
5,0.013593,0.009578,0.005595,7.998471e-04,auto,minkowski,100,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.808036,0.835985,0.747056,0.890114,0.847662,0.825770,0.047401,13
6,0.009594,0.004584,0.010395,2.573906e-03,ball_tree,minkowski,10,2,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.855314,0.874254,0.858503,0.946663,0.964286,0.899804,0.046242,1
7,0.017989,0.013393,0.004597,1.742154e-03,ball_tree,minkowski,10,2,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.855314,0.874254,0.859435,0.929144,0.964286,0.896486,0.042955,5
8,0.014192,0.010955,0.011192,2.225394e-03,ball_tree,minkowski,50,2,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.729723,0.723577,0.588177,0.830519,0.708333,0.716066,0.077146,17
9,0.007200,0.000745,0.004394,4.883702e-04,ball_tree,minkowski,50,2,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.835985,0.855314,0.802399,0.910254,0.888158,0.858422,0.038007,9


In [64]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.94      0.94        79
           1       0.88      0.90      0.89        41

    accuracy                           0.93       120
   macro avg       0.91      0.92      0.92       120
weighted avg       0.93      0.93      0.93       120

